In [1]:
from apex import *
from proofs import *
from perf_data import *

In [2]:
z = ZigZag(security=proofs.filecoin_security_requirements, partitions=8, size=64*GiB, merkle_hash=pedersen)

In [3]:
(l1, savings, savings_, best_) = optimize(z); (l, savings, savings_, best_)

In [4]:
a = apex(z, l); a

In [5]:
zz = ZigZag(security=proofs.filecoin_security_requirements, partitions=8, size=64*GiB, merkle_hash=blake2s)

In [6]:
(ll, savings, savings_, best_) = optimize(zz); (ll, savings, savings_, best_)

In [7]:
aa = apex(zz, ll); aa

ZigZag(instance=None, circuit_proof_size=192, hash_size=32, size=68719476736, partitions=8, merkle_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), kdf_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), security=Security(base_degree=5, expansion_degree=8, layers=10, total_challenges=8848, sloth_iter=0), merkle_pessimization=1.5, processor_power=165, relax_time=1, constraint_proving_time=5.013000000000001e-05, apex_height=17)

In [8]:
z.show_times()

groth: 02:49:18.2; replication: 181:27:10.4; seal: 184:16:28.5


In [9]:
a.show_times()

groth: 02:52:34.9; replication: 181:26:50.9; seal: 184:19:25.8


In [10]:
a.groth_proving_time()/z.groth_proving_time()

1.0193681388429139

In [11]:
zz.show_times()

groth: 22:41:27.2; replication: 05:22:50.8; seal: 28:04:18.0


In [12]:
aa.show_times()

groth: 12:11:24.1; replication: 05:22:50.7; seal: 17:34:14.8


In [13]:
aa.groth_proving_time()/zz.groth_proving_time()

0.5372218364641622

In [14]:
zz.constraints()

5560733528.0

In [15]:
aa.constraints()

2987347478.0

In [16]:
zz.constraints() - aa.constraints()

2573386050.0

In [17]:
aa.constraints() / zz.constraints()

0.5372218364641622

In [18]:
projected = ZigZag(security=proofs.filecoin_security_requirements, instance=projected_instance, partitions=8)

In [19]:
optimize(projected)

(13, 11395.223248500028, '03:09:55.2', '163:17:59.6')

In [20]:
apex_projected = apex(projected, 13)

In [21]:
apex_projected.show_times(64*GiB)

groth: 09:20:59.4; replication: 153:56:28.4; seal: 163:17:59.6


In [22]:
apex_projected.meets_performance_requirements(64*GiB, filecoin_scaling_requirements)

False

In [23]:
projected_blake = projected.scaled_for_new_hash(blake2s)

In [24]:
optimize(projected_blake)

(17, 129003.84268649993, '35:50:3.8', '177:46:37.6')

In [25]:
apex_projected_blake = apex(projected_blake, 17)

In [26]:
apex_projected_blake.show_times(64 * GiB)

groth: 141:12:30.5; replication: 36:33:35.3; seal: 177:46:37.6


In [27]:
projected_blake.show_times(64 * GiB)

groth: 177:02:34.3; replication: 36:33:35.3; seal: 213:36:41.5


In [28]:
projected_blake.meets_performance_requirements(64*GiB, filecoin_scaling_requirements)

False

In [29]:
(l, _, savings, best) = optimize(projected_blake); (l, savings, best)

(17, '35:50:3.8', '177:46:37.6')

In [30]:
apex_projected_blake = apex(projected_blake, 17)

In [31]:
apex_projected_blake.meets_performance_requirements(64*GiB, filecoin_scaling_requirements)

False

### OUTDATED
**Yes!** This will meet scaling requirements. We need to consider how large the circuits are. We also need to repartition so the first layer of challenges is a power of 2.

In [32]:
apex_projected_blake.constraints() / apex_projected_blake.partitions # how many constraints in each partition

12585041317.61111

This is huge. So we will need to go 128-GiB sectors. However, that means we can go up to 16 partitions. Verify that: (TODO — should be a simpler way to ask for only this.)~

Actually I failed to divide by partitions the first time, so the below is unncessary.

In [33]:
ZigZag(partitions=16).proof_size() <= filecoin_scaling_requirements.proof_size * 128 * GiB

True

Let's work through this again with 128 GiB sectors:

In [34]:
projected128 = ZigZag(security=proofs.filecoin_security_requirements, instance=projected_instance, partitions=8)

In [35]:
simulated128 = ZigZag(security=proofs.filecoin_security_requirements, partitions=8, size=128*GiB, merkle_hash=blake2s)

In [36]:
projected.constraints(), projected128.constraints(), simulated128.constraints()

(7037149056, 7037149056, 5732008688.0)

In [37]:
optimize(projected128)

(13, 11395.223248500028, '03:09:55.2', '163:17:59.6')

In [38]:
apex_projected128 = apex(projected128, 13)

In [39]:
apex_projected128

ZigZag(instance=Instance(encoding_replication_time_per_GiB=2018, sector_size=68719476736, constraints=879643632, groth_proving_time=44096.53527216001, machine=Machine(clock_speed_ghz=2.3, ram_gb=3904, cores=64, hourly_cost=None), description='x1e32.xlarge projected', merkle_tree_hash=HashFunction(hash_time=1.7993e-05, constraints=1152), layers=10, vanilla_proving_time=31.808, security=Security(base_degree=5, expansion_degree=8, layers=10, total_challenges=8848, sloth_iter=0)), circuit_proof_size=192, hash_size=32, size=68719476736, partitions=8, merkle_hash=HashFunction(hash_time=1.7993e-05, constraints=1152), kdf_hash=HashFunction(hash_time=1.0428e-07, constraints=10324), security=Security(base_degree=5, expansion_degree=8, layers=10, total_challenges=8848, sloth_iter=0), merkle_pessimization=1.5, processor_power=165, relax_time=1, constraint_proving_time=5.013000000000001e-05, apex_height=13)

In [40]:
humanize_bytes(apex_projected128.groth_proving_memory())

'1.2 TB'

In [41]:
apex_projected128.meets_performance_requirements(128*GiB, filecoin_scaling_requirements)

False

In [42]:
projected_blake128 = projected128.scaled_for_new_hash(blake2s)

In [43]:
optimize(projected_blake128)

(17, 129003.84268649993, '35:50:3.8', '177:46:37.6')

In [44]:
apex_projected_blake128 = apex(projected_blake128, 17)

In [45]:
apex_projected_blake128.meets_performance_requirements(128*GiB, filecoin_scaling_requirements)

True

Still good.

In [46]:
(apex_projected_blake128.constraints(), apex_projected_blake.constraints())

(100680330540.88889, 100680330540.88889)

In [47]:
apex_projected_blake128.merkle_time(100)

523.833311232

In [48]:
apex_projected_blake128.replication_time()

2056.48961728502

In [49]:
apex_projected_blake128.merkle_pessimization=100

In [50]:
apex_projected_blake128.meets_performance_requirements(128*GiB, filecoin_scaling_requirements)

True

In [51]:
apex_projected_blake128.replication_time()

2056.48961728502

In [52]:
apex_projected_blake128.show_times()

groth: 141:12:30.5; replication: 36:33:35.3; seal: 177:46:37.6


In [53]:
apex_projected_blake128.constraints() * constraint_ram

156076134285076.22

In [54]:
apex_projected_blake128.partition_constraints()

12585041317.61111

In [55]:
humanize_bytes(apex_projected_blake128.partition_constraints() * constraint_ram)

'17.7 TB'

In [56]:
humanize_bytes((apex_projected_blake128.constraints() / 32) * constraint_ram)

'4.4 TB'